In [1]:
# 이 부분은 처음 한번만 실행하면 됌.
# 코드 수정 - "The reason is that the last Ubuntu update update supports chromium driver just via snap."
# 최근 우분투 업데이트에서 크롬 드라이버 설치를 snap을 이용해서만 하도록 바뀜
# 고로 snap 없이 설치하는 아래 우회 코드로 변경
# 출처 : https://colab.research.google.com/drive/1cbEvuZOhkouYLda3RqiwtbM-o9hxGLyC
# 출처2 : https://stackoverflow.com/questions/75155063/selenium-use-chrome-on-colab-got-unexpectedly-exited

%%shell
# Ubuntu no longer distributes chromium-browser outside of snap
#
# Proposed solution: https://askubuntu.com/questions/1204571/how-to-install-chromium-without-snap

# Add debian buster
cat > /etc/apt/sources.list.d/debian.list <<'EOF'
deb [arch=amd64 signed-by=/usr/share/keyrings/debian-buster.gpg] http://deb.debian.org/debian buster main
deb [arch=amd64 signed-by=/usr/share/keyrings/debian-buster-updates.gpg] http://deb.debian.org/debian buster-updates main
deb [arch=amd64 signed-by=/usr/share/keyrings/debian-security-buster.gpg] http://deb.debian.org/debian-security buster/updates main
EOF

# Add keys
apt-key adv --keyserver keyserver.ubuntu.com --recv-keys DCC9EFBF77E11517
apt-key adv --keyserver keyserver.ubuntu.com --recv-keys 648ACFD622F3D138
apt-key adv --keyserver keyserver.ubuntu.com --recv-keys 112695A0E562B32A

apt-key export 77E11517 | gpg --dearmour -o /usr/share/keyrings/debian-buster.gpg
apt-key export 22F3D138 | gpg --dearmour -o /usr/share/keyrings/debian-buster-updates.gpg
apt-key export E562B32A | gpg --dearmour -o /usr/share/keyrings/debian-security-buster.gpg

# Prefer debian repo for chromium* packages only
# Note the double-blank lines between entries
cat > /etc/apt/preferences.d/chromium.pref << 'EOF'
Package: *
Pin: release a=eoan
Pin-Priority: 500


Package: *
Pin: origin "deb.debian.org"
Pin-Priority: 300


Package: chromium*
Pin: origin "deb.debian.org"
Pin-Priority: 700
EOF

# Install chromium and chromium-driver
apt-get update
apt-get install chromium chromium-driver

# Install selenium
pip install selenium

Executing: /tmp/apt-key-gpghome.2GBLFO3x5M/gpg.1.sh --keyserver keyserver.ubuntu.com --recv-keys DCC9EFBF77E11517
gpg: key DCC9EFBF77E11517: public key "Debian Stable Release Key (10/buster) <debian-release@lists.debian.org>" imported
gpg: Total number processed: 1
gpg:               imported: 1
Executing: /tmp/apt-key-gpghome.arEF2lc3pO/gpg.1.sh --keyserver keyserver.ubuntu.com --recv-keys 648ACFD622F3D138
gpg: key DC30D7C23CBBABEE: public key "Debian Archive Automatic Signing Key (10/buster) <ftpmaster@debian.org>" imported
gpg: Total number processed: 1
gpg:               imported: 1
Executing: /tmp/apt-key-gpghome.Uji9OU2C68/gpg.1.sh --keyserver keyserver.ubuntu.com --recv-keys 112695A0E562B32A
gpg: key 4DFAB270CAA96DFA: public key "Debian Security Archive Automatic Signing Key (10/buster) <ftpmaster@debian.org>" imported
gpg: Total number processed: 1
gpg:               imported: 1
Get:1 http://deb.debian.org/debian buster InRelease [122 kB]
Get:2 http://security.ubuntu.com/ubuntu

In [2]:
!which chromium-browser
!which chromium

/usr/bin/chromium


ERD 기반 데이터 배치 및 전처리

In [3]:
import csv
import os

TOP_CATEGORY_CODE = "00"
input_csv = "/content/카카오맵 가게_지점 리스트 크롤러(판교 중식).csv"
output_csv = "chinese_restaurants_test.csv"

INCLUDE_REVIEW = True

if INCLUDE_REVIEW:
    final_columns = {
        "식당 id": None,   # 새로 생성
        "식당이름": 1,
        "카테고리": 0,
        "평점": 3,
        "리뷰수": 5,
        "식당주소": 6,
        "운영시간": 8,
        "전화번호": 9
    }
else:
    final_columns = {
        "식당 id": None,
        "식당이름": 1,
        "카테고리": 0,
        "평점": 3,
        "식당주소": 6,
        "운영시간": 8,
        "전화번호": 9
    }

with open(input_csv, newline='', encoding="utf-8") as infile, \
     open(output_csv, "w", newline="", encoding="utf-8") as outfile:

    reader = csv.reader(infile)
    writer = csv.writer(outfile)

    header = next(reader)  # 원본 CSV 첫 줄 건너뛰기 (헤더 가정)

    new_header = list(final_columns.keys())
    writer.writerow(new_header)

    row_count = 0

    for row in reader:
        row_count += 1
        new_row = []

        for col_name, idx in final_columns.items():
            if col_name == "식당 id" and idx is None:
                custom_id = f"{TOP_CATEGORY_CODE}-{row_count:02d}"
                new_row.append(custom_id)
            else:
                # 일단 문자열로 강제 변환
                original_value = str(row[idx]).strip() if idx < len(row) else ""

                if col_name == "카테고리":
                    # '판교' 제거
                    new_value = original_value.replace("판교", "").strip()
                else:
                    new_value = original_value

                new_row.append(new_value)

        writer.writerow(new_row)

print("CSV 전처리 및 저장 완료:", os.path.abspath(output_csv))


CSV 전처리 및 저장 완료: /content/chinese_restaurants_test.csv


식당정보 리스트 크롤러 (ERD 기반)

In [11]:
import csv
import os
import sys
import time
import json
import uuid
from selenium import webdriver
from selenium.webdriver.common.keys import Keys   # 키 입력 (ENTER 등)
from selenium.webdriver.common.by import By       # 요소 찾기 기준 제공
from selenium.webdriver.chrome.service import Service  # Service 클래스 임포트
from selenium.webdriver.support.ui import WebDriverWait  # Explicit wait
from selenium.webdriver.support import expected_conditions as EC

# ---------------------------
# 1. 전처리된 CSV 파일 읽기
# ---------------------------
# 전처리된 CSV 파일은 아래와 같이 구성되어 있습니다:
# [검색키워드, 가게이름, 카테고리, 평점, 주소, 영업시간, 연락처, 상세정보링크]
input_csv = "/content/chinese_restaurants_test.csv"

# CSV 파일을 읽어 각 행을 리스트에 저장합니다.
restaurants_list = []
with open(input_csv, newline='', encoding="utf-8") as infile:
    reader = csv.reader(infile)
    header = next(reader)  # 헤더 읽기 (참고용)
    for row in reader:
        # row: [검색키워드, 가게이름, 카테고리, 평점, 주소, 영업시간, 연락처, 상세정보링크]
        if len(row) >= 7:
            restaurants_list.append(row)

print("총 레코드 수:", len(restaurants_list))
print("첫 레코드:", restaurants_list[0])

# ---------------------------
# 2. Selenium 크롤링 설정
# ---------------------------
# 추가 크롤링 정보:
# - 사진 (image_urls): 클래스 "link_photo" → 최대 3개 (style 속성의 background-image URL)
# - 편의시설 (convenience): 클래스 "list_facility"의 텍스트
# - 유의사항 (caution): 예약, 배달, 포장 정보를 나타내는 섹션에서 추출
#   (예약가능, 예약불가, 배달가능, 배달불가, 포장가능, 포장불가 중 해당하는 값만 추출)
options = webdriver.ChromeOptions()
options.add_argument('--headless')              # Headless 모드
options.add_argument('--no-sandbox')
options.add_argument('--disable-dev-shm-usage')
options.add_experimental_option("excludeSwitches", ["enable-logging"])
if os.path.exists('/usr/bin/chromium-browser'):
    options.binary_location = '/usr/bin/chromium-browser'
else:
    options.binary_location = '/usr/bin/chromium'

service = Service(executable_path='/usr/bin/chromedriver')
driver = webdriver.Chrome(service=service, options=options)

# ---------------------------
# 3. 크롤링 수행
# ---------------------------
# 최종 데이터를 저장할 리스트 (각 항목은 딕셔너리)
final_data = []

for row in restaurants_list:
    # CSV 열 인덱스 정의
    # 0: 식당 id
    # 1: 식당이름
    # 2: 카테고리
    # 3: 평점
    # 4: 리뷰수
    # 5: 식당주소
    # 6: 운영시간
    # 7: 전화번호

    id             = row[0]  # 식당id
    store_name     = row[1]  # 식당이름
    category_id    = row[2]  # 카테고리
    score          = row[3]  # 평점
    review         = row[4]  # 리뷰수
    address        = row[5]  # 주소
    operating_hour = row[6]  # 영업시간
    phone_number   = row[7]  # 전화번호

    final_row = {
        "id": id,
        "category_id": category_id,
        "name": store_name,
        "score": score,
        "review": review,
        "address": address,
        "operating_hour": operating_hour,
        "phone_number": phone_number,
        "image_urls": "",
        "convenience": "",
        "caution": "",
        "is_deleted": False
    }

    print("크롤링 시작:", store_name, end=" ")
    try:
        # 상세 페이지 접속: 여기서는 가게 이름을 검색어로 사용
        search_url = f"https://map.kakao.com/?q={store_name}"
        driver.get(search_url)
        driver.implicitly_wait(3)

        # 상세보기 버튼 클릭: 절대 XPath 사용
        try:
            detail_link = WebDriverWait(driver, 10).until(
                EC.presence_of_element_located(
                    (By.XPATH, '//*[@id="info.search.place.list"]/li/div[5]/div[4]/a[1]')
                )
            )
            # 클릭 효과를 위해 ENTER 키 전송
            detail_link.send_keys(Keys.ENTER)
            driver.switch_to.window(driver.window_handles[-1])
        except Exception as e:
            print("상세보기 링크를 찾을 수 없음:", e)
            final_row["image_urls"] = "정보 없음"
            final_row["convenience"] = "정보 없음"
            final_row["caution"] = "정보 없음"
            final_data.append(final_row)
            continue

        time.sleep(1)  # 페이지 로딩 안정화

        # (A) 사진 정보 추출 (클래스: link_photo) → 최대 3개
        try:
            photo_elements = driver.find_elements(By.CLASS_NAME, "link_photo")
            image_urls = []
            for elem in photo_elements:
                style_attr = elem.get_attribute("style")
                if style_attr and "url(" in style_attr:
                    url_part = style_attr.split("url(")[1].split(")")[0].strip("\"'")
                    image_urls.append(url_part)
                if len(image_urls) >= 3:
                    break
            if not image_urls:
                image_urls = "이미지 정보 없음"
        except Exception as e:
            print("사진 정보 추출 에러:", e)
            image_urls = "이미지 정보 없음"
        final_row["image_urls"] = image_urls

        # (B) 편의시설 정보 추출 (클래스: list_facility)
        try:
            facility_elem = driver.find_element(By.CLASS_NAME, "list_facility")
            facility_text = facility_elem.text.strip()
            if not facility_text:
                facility_text = "편의시설 정보 없음"
        except Exception as e:
            print("편의시설 정보 추출 에러:", e)
            facility_text = "편의시설 정보 없음"
        final_row["convenience"] = facility_text

        # (C) 유의사항 (예약, 배달, 포장 정보) 추출
        # 예약/배달/포장 정보는 해당 섹션에 존재하며, 예를 들어:
        # <div class="placeinfo_default">
        #     <h4 class="tit_detail"><span class="ico_comm ico_delivery">예약, 배달, 포장</span></h4>
        #     <div class="location_detail">예약가능, 배달불가, 포장가능</div>
        # </div>
        try:
            reservation_elem = driver.find_element(By.XPATH,
                "//div[contains(@class, 'placeinfo_default')]//h4[.//span[contains(@class, 'ico_delivery')]]/following-sibling::div[contains(@class, 'location_detail')]"
            )
            reservation_text = reservation_elem.text.strip()
            # 가능한 키워드
            valid_keywords = {"예약가능", "예약불가", "배달가능", "배달불가", "포장가능", "포장불가"}
            # 쉼표로 구분되어 있을 경우 분리
            tokens = [token.strip() for token in reservation_text.split(",")]
            reservation_info = ", ".join([token for token in tokens if token in valid_keywords])
            if not reservation_info:
                reservation_info = "유의사항 정보 없음"
        except Exception as e:
            print("예약/배달/포장 정보 추출 에러:", e)
            reservation_info = "유의사항 정보 없음"
        final_row["caution"] = reservation_info

        print("상세 정보 저장 완료.")

        # 상세 페이지 창 닫기 후 원래 창으로 전환
        driver.close()
        driver.switch_to.window(driver.window_handles[0])
    except Exception as e:
        print("크롤링 오류, 에러:", e)
        final_row["image_urls"] = "정보 없음"
        final_row["convenience"] = "정보 없음"
        final_row["caution"] = "정보 없음"
        if len(driver.window_handles) > 1:
            driver.close()
            driver.switch_to.window(driver.window_handles[0])
    print()
    final_data.append(final_row)

driver.quit()

# ---------------------------
# 4. 최종 CSV 파일로 저장
# 최종 CSV의 열 순서:
fieldnames = [
    "id",             # 식당 id
    "name",           # 식당이름
    "category_id",    # 카테고리
    "score",          # 평점
    "review",         # 리뷰수
    "address",        # 식당주소
    "operating_hour", # 운영시간
    "phone_number",   # 전화번호
    "image_urls",
    "convenience",
    "caution",
    "is_deleted"
]

# 1) category_id를 사용한 CSV 파일 경로
csv_output_path_1 = f"{category_id}_restaurants_table.csv"

# 2) 고정된 CSV 파일명 (final_restaurants_table.csv)
csv_output_path_2 = "final_restaurants_table.csv"

# ---------------------------
# 두 개의 CSV 파일을 각각 생성
# ---------------------------

# (A) 첫 번째 CSV: f"{category_id}_restaurants_table.csv" - BE 공유용
with open(csv_output_path_1, "w", newline="", encoding="utf-8") as csvfile1:
    writer1 = csv.DictWriter(csvfile1, fieldnames=fieldnames)
    writer1.writeheader()

    for row in final_data:
        # image_urls가 리스트이면 문자열로 변환
        if isinstance(row.get("image_urls"), list):
            row["image_urls"] = ", ".join(row["image_urls"])
        writer1.writerow(row)

print("첫 번째 CSV 파일 저장 완료:", os.path.abspath(csv_output_path_1))

# (B) 두 번째 CSV: "final_restaurants_table.csv" - 메뉴 데이터 크롤링용
with open(csv_output_path_2, "w", newline="", encoding="utf-8") as csvfile2:
    writer2 = csv.DictWriter(csvfile2, fieldnames=fieldnames)
    writer2.writeheader()

    for row in final_data:
        # image_urls가 리스트이면 문자열로 변환
        if isinstance(row.get("image_urls"), list):
            row["image_urls"] = ", ".join(row["image_urls"])
        writer2.writerow(row)

print("두 번째 CSV 파일 저장 완료:", os.path.abspath(csv_output_path_2))

# ---------------------------
# 5. JSON 파일로도 저장
# ---------------------------
json_output_path = f"{category_id}_restaurants_table.json"
with open(json_output_path, "w", encoding="utf-8") as jsonfile:
    # ensure_ascii=False → 한글이 유니코드 이스케이프(\uAC00)로 안 나오게
    # indent=4 → 보기 좋은 포맷
    json.dump(final_data, jsonfile, ensure_ascii=False, indent=4)

print("JSON 파일 저장 완료:", os.path.abspath(json_output_path))


총 레코드 수: 59
첫 레코드: ['00-01', '이가네양꼬치 판교본점', '중식', '4.0', '570', '경기 성남시 분당구 분당내곡로 155 KCC웰츠타워 104호', '매일 12:00 ~ 24:00', '050713599688']
크롤링 시작: 이가네양꼬치 판교본점 상세 정보 저장 완료.

크롤링 시작: 동청담 삼평직영점 상세 정보 저장 완료.

크롤링 시작: 차알 판교점 상세 정보 저장 완료.

크롤링 시작: 몽중헌 판교점 상세 정보 저장 완료.

크롤링 시작: 최고집짬뽕 상세 정보 저장 완료.

크롤링 시작: 청계산수타 상세 정보 저장 완료.

크롤링 시작: 신승반점 현대백화점판교점 상세 정보 저장 완료.

크롤링 시작: 베이징스토리 상세 정보 저장 완료.

크롤링 시작: 마라공방 판교점 상세 정보 저장 완료.

크롤링 시작: 명품조박사짬뽕짜장 상세 정보 저장 완료.

크롤링 시작: 전국5대짬뽕연화산 상세 정보 저장 완료.

크롤링 시작: 왕스덕 상세 정보 저장 완료.

크롤링 시작: 공화춘 현대백화점판교점 예약/배달/포장 정보 추출 에러: Message: no such element: Unable to locate element: {"method":"xpath","selector":"//div[contains(@class, 'placeinfo_default')]//h4[.//span[contains(@class, 'ico_delivery')]]/following-sibling::div[contains(@class, 'location_detail')]"}
  (Session info: headless chrome=90.0.4430.212); For documentation on this error, please visit: https://www.selenium.dev/documentation/webdriver/troubleshooting/errors#no-such-element-exception
Stacktrace:
#0 0x571d28911

식당 메뉴 크롤러 (ERD 기반)

In [ ]:
import csv
import os
import time
import json
import uuid

from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC

# ===============================================
# Step 1: CSV 파일에서 식당 id, name 매핑 생성
# ===============================================
restaurants_csv = "final_restaurants_table.csv"
restaurants = []
with open(restaurants_csv, newline='', encoding="utf-8") as infile:
    reader = csv.DictReader(infile)
    for row in reader:
        if row.get("id") and row.get("name") and row.get("category_id"):
            restaurants.append({
                "id": row["id"],
                "name": row["name"],
                "category_id": row["category_id"]
            })

print("총 식당 수:", len(restaurants))
if restaurants:
    print("샘플 식당:", restaurants[0])

# ===============================================
# Step 2: Selenium 크롤링 설정
# ===============================================
options = webdriver.ChromeOptions()
options.add_argument('--headless')  # Headless 모드
options.add_argument('--no-sandbox')
options.add_argument('--disable-dev-shm-usage')
options.add_experimental_option("excludeSwitches", ["enable-logging"])

if os.path.exists('/usr/bin/chromium-browser'):
    options.binary_location = '/usr/bin/chromium-browser'
else:
    options.binary_location = '/usr/bin/chromium'

service = Service(executable_path='/usr/bin/chromedriver')
driver = webdriver.Chrome(service=service, options=options)

# ===============================================
# Step 3: 메뉴 정보 크롤링 (더보기 버튼 클릭 로직 포함)
# ===============================================
menu_records = []

# menu_id를 1부터 시작
menu_id_counter = 1

for restaurant in restaurants:
    rest_id = restaurant["id"]       # CSV에서 읽어온 식당 id
    rest_name = restaurant["name"]

    print("=== 크롤링 시작:", rest_name, "===")

    try:
        # (1) 검색 페이지 열기
        search_url = f"https://map.kakao.com/?q={rest_name}"
        driver.get(search_url)
        driver.implicitly_wait(3)

        # (2) 상세보기 버튼 클릭
        try:
            detail_link = WebDriverWait(driver, 10).until(
                EC.presence_of_element_located(
                    (By.XPATH, '//*[@id="info.search.place.list"]/li/div[5]/div[4]/a[1]')
                )
            )
            detail_link.send_keys(Keys.ENTER)
            driver.switch_to.window(driver.window_handles[-1])
        except Exception as e:
            print("  상세보기 링크를 찾을 수 없음:", e)
            continue

        time.sleep(2)

        # (3) "더보기" 버튼이 있으면 클릭
        try:
            more_button_xpath = '//*[@id="mArticle"]/div[3]/a'
            more_button = WebDriverWait(driver, 3).until(
                EC.element_to_be_clickable((By.XPATH, more_button_xpath))
            )
            if more_button.is_displayed():
                more_button.click()
                time.sleep(1)
                print("  '더보기' 버튼 클릭 완료.")
        except Exception as e:
            # 더보기 버튼이 없거나, 이미 메뉴가 전부 노출된 경우
            print("  '더보기' 버튼 없음 or 클릭 불필요:", e)

        # (4) 메뉴 정보 추출
        try:
            menulist = driver.find_element(By.CLASS_NAME, 'list_menu')
            menus = menulist.find_elements(By.CLASS_NAME, 'loss_word')
            menus_price = menulist.find_elements(By.CLASS_NAME, 'price_menu')

            extracted_menu = {}
            for m, p in zip(menus, menus_price):
                menu_text = m.text.strip()
                price_text = p.text.strip()

                if price_text.startswith("가격:"):
                    price_text = price_text.replace("가격:", "").strip()

                if menu_text and price_text:
                    extracted_menu[menu_text] = price_text
        except Exception as e:
            print(" 메뉴 정보 추출 에러:", e)
            extracted_menu = {}

        if not extracted_menu:
            print(" 메뉴 정보 없음.")
        else:
            # 각 메뉴에 대해 순회
            for menu_name, price_str in extracted_menu.items():
                # 쉼표 제거 후 int 변환 (실패 시 0)
                try:
                    price_int = int(price_str.replace(",", "").strip())
                except:
                    price_int = 0

                # 'menu_id_counter'를 사용해 menu_id 할당
                record = {
                    "restaurant_id": rest_id,
                    "menu_id": menu_id_counter,   # 여기서부터 1씩 증가
                    "menu_name": menu_name,
                    "price": price_int,
                    "is_deleted": False,
                    "is_active": True
                }
                menu_records.append(record)

                # 다음 메뉴를 위해 +1
                menu_id_counter += 1

            print(f"  {rest_name} 메뉴 {len(extracted_menu)}개 → 기록 생성 완료.")

        # (5) 상세 페이지 창 닫기
        driver.close()
        driver.switch_to.window(driver.window_handles[0])

    except Exception as e:
        print("  오류 발생:", e)
        if len(driver.window_handles) > 1:
            driver.close()
            driver.switch_to.window(driver.window_handles[0])
        continue

    print()

driver.quit()

# ===============================================
# Step 4: JSON & CSV 파일로 결과 저장
# ===============================================
# 4-1) JSON
output_json = f"{category_id}_menus_table.json"
with open(output_json, "w", encoding="utf-8") as f:
    json.dump(menu_records, f, ensure_ascii=False, indent=4)
print("메뉴 JSON 파일 저장 완료:", os.path.abspath(output_json))

output_json = f"{category_id}_menus_table.json"

# 4-2) CSV
# ※ 열 순서를 "restaurant_id" → "menu_id" → "menu_name" → "price" → "is_deleted" → "is_active" 로 바꿉니다.
fieldnames = [
    "restaurant_id",
    "menu_id",
    "menu_name",
    "price",
    "is_deleted",
    "is_active"
]

with open(output_csv, "w", newline="", encoding="utf-8") as f:
    writer = csv.DictWriter(f, fieldnames=fieldnames)
    writer.writeheader()
    for rec in menu_records:
        writer.writerow(rec)

output_csv = f"{category_id}_menus_table.csv"


총 식당 수: 59
샘플 식당: {'id': '00-01', 'name': '이가네양꼬치 판교본점', 'category_id': '중식'}
=== 크롤링 시작: 이가네양꼬치 판교본점 ===
  '더보기' 버튼 클릭 완료.
  이가네양꼬치 판교본점 메뉴 12개 → 기록 생성 완료.

=== 크롤링 시작: 동청담 삼평직영점 ===
  '더보기' 버튼 클릭 완료.
  동청담 삼평직영점 메뉴 20개 → 기록 생성 완료.

=== 크롤링 시작: 차알 판교점 ===
  '더보기' 버튼 없음 or 클릭 불필요: Message: 
Stacktrace:
#0 0x5a42a8cc37f9 <unknown>
#1 0x5a42a8c633b3 <unknown>
#2 0x5a42a89ab016 <unknown>
#3 0x5a42a89df81e <unknown>
#4 0x5a42a8a158fb <unknown>
#5 0x5a42a8a02ded <unknown>
#6 0x5a42a8a139e1 <unknown>
#7 0x5a42a8a02c93 <unknown>
#8 0x5a42a89d4ce4 <unknown>
#9 0x5a42a89d64d2 <unknown>
#10 0x5a42a8c8f542 <unknown>
#11 0x5a42a8c9ece7 <unknown>
#12 0x5a42a8c9e9e4 <unknown>
#13 0x5a42a8ca313a <unknown>
#14 0x5a42a8c9f5b9 <unknown>
#15 0x5a42a8c84e00 <unknown>
#16 0x5a42a8cb65d2 <unknown>
#17 0x5a42a8cb6778 <unknown>
#18 0x5a42a8ccea1f <unknown>
#19 0x7a43a763fac3 <unknown>
#20 0x7a43a76d1850 <unknown>

  차알 판교점 메뉴 5개 → 기록 생성 완료.

=== 크롤링 시작: 몽중헌 판교점 ===
  '더보기' 버튼 클릭 완료.
  몽중헌 판교점 메뉴 21개 → 기록 생성 완